In [2]:
import pandas as pd
df = pd.read_csv("C:/Users/joeco/Python/fb-ref-data/data/181220_2021_aggregated_player_data.csv")
df.head()

,team,name,matches,pos,age,total_mins,goals,assists,pens_made,pens_attempted,...,miscontrols,dispossessed,fouls,times_fouled,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost
0,Southampton,Jake Vokins,1,LB,20,66,0,0,0,0,...,0,0,2,0,0,0,0,2,0,0
1,Sheffield United,Rhian Brewster,17,"FW,LW",20,733,0,0,0,0,...,13,14,12,10,0,0,0,40,6,33
2,Aston Villa,Conor Hourihane,4,CM,29,225,1,1,0,0,...,1,0,1,0,0,0,0,12,3,0
3,Wolverhampton Wanderers,Vitinha,12,DM,20,232,0,1,0,0,...,4,4,8,5,0,0,0,15,0,1
4,Sheffield United,Max Lowe,7,WB,23,460,0,0,0,0,...,3,5,10,2,0,0,0,35,5,16


In [3]:
def create_per_90_stat(row,stat_col):
    total_mins = int(row["total_mins"])
    stat = round(float(row[stat_col]),2)
    total_90s = round(float(total_mins/90),2)
    per_90s_stat = round(float(stat/total_90s),2)
    return per_90s_stat

def isAboveMin(row,stat,min_base):
    value = round(float(row[stat]),2)
    if value >= float(min_base):
        return True
    return False

def top_10_for_stat(df, stat, min_base, other_stats_to_keep=[], per90=True, west_ham_only=False):
    df["keep"] = df.apply(isAboveMin,args=(stat,min_base),axis=1)
    df = df.query("keep == True & total_mins >= 90")
    relevant_data = list(set(["team","name",stat,"total_mins"] + other_stats_to_keep))
    df = df[relevant_data]
    if west_ham_only:
        a = "West Ham United"
        df = df.query("team == @a")
    if per90:
        p90_name = stat + "_p90"
        df[p90_name] = df.apply(create_per_90_stat,args=(stat,),axis=1)
        sorted_df = df.sort_values(by=[p90_name,stat],ascending=False)
        top_10 = sorted_df[["team","name",stat,p90_name,"total_mins"] + other_stats_to_keep].head(10)
    else:
        sorted_df = df.sort_values(by=[stat],ascending=False)
        top_10 = sorted_df[["team","name",stat,"total_mins"] + other_stats_to_keep].head(10)
    return top_10   

In [3]:
df_miscontrols = top_10_for_stat(df,"miscontrols",2,per90=False)
df_miscontrols

,team,name,miscontrols,total_mins
164,Liverpool,Mohamed Salah,66,1742
483,Aston Villa,Ollie Watkins,63,1800
457,Everton,Dominic Calvert-Lewin,63,1672
234,Newcastle United,Callum Wilson,57,1777
323,Crystal Palace,Wilfried Zaha,56,1667
123,Fulham,Ademola Lookman,52,1481
430,Liverpool,Sadio Mané,52,1600
422,Wolverhampton Wanderers,Pedro Neto,51,1803
356,Southampton,Che Adams,50,1623
146,Leeds United,Jack Harrison,50,1665


In [6]:
top_10_for_stat(df,"dispossessed",15)

,team,name,dispossessed,dispossessed_p90,total_mins
323,Crystal Palace,Wilfried Zaha,76,4.10,1667
331,Crystal Palace,Jordan Ayew,44,3.75,1055
188,Newcastle United,Allan Saint-Maximin,28,3.70,681
221,Leeds United,Hélder Costa,26,3.37,694
215,Crystal Palace,Jeffrey Schlupp,36,3.32,976
122,Manchester City,Raheem Sterling,55,3.24,1529
197,West Ham United,Michail Antonio,35,3.16,995
14,Crystal Palace,Michy Batshuayi,21,3.07,615
332,Tottenham Hotspur,Giovani Lo Celso,15,3.07,439
164,Liverpool,Mohamed Salah,59,3.05,1742


In [7]:
top_10_for_stat(df,"dispossessed",15,per90=False)

,team,name,dispossessed,total_mins
323,Crystal Palace,Wilfried Zaha,76,1667
164,Liverpool,Mohamed Salah,59,1742
122,Manchester City,Raheem Sterling,55,1529
430,Liverpool,Sadio Mané,52,1600
456,Fulham,Andre-Frank Zambo Anguissa,48,1677
234,Newcastle United,Callum Wilson,45,1777
331,Crystal Palace,Jordan Ayew,44,1055
420,West Ham United,Jarrod Bowen,43,1621
338,Leicester City,Harvey Barnes,43,1625
123,Fulham,Ademola Lookman,43,1481


In [9]:
top_10_for_stat(df,"tackles",15)

,team,name,tackles,tackles_p90,total_mins
450,Leicester City,Wilfred Ndidi,46,4.80,862
125,Fulham,Kenny Tete,25,4.44,507
405,Tottenham Hotspur,Serge Aurier,45,4.41,918
4,Sheffield United,Max Lowe,21,4.11,460
78,Crystal Palace,Tyrick Mitchell,40,3.64,990
370,Southampton,Oriol Romeu,56,3.62,1393
380,Tottenham Hotspur,Pierre Højbjerg,72,3.60,1800
390,Manchester United,Aaron Wan-Bissaka,71,3.55,1800
332,Tottenham Hotspur,Giovani Lo Celso,17,3.48,439
196,Fulham,Mario Lemina,23,3.39,610


In [10]:
top_10_for_stat(df,"tackles",15,per90=False)

,team,name,tackles,total_mins
380,Tottenham Hotspur,Pierre Højbjerg,72,1800
390,Manchester United,Aaron Wan-Bissaka,71,1800
114,Leicester City,James Justin,60,1980
381,Brighton & Hove Albion,Yves Bissouma,59,1673
456,Fulham,Andre-Frank Zambo Anguissa,59,1677
370,Southampton,Oriol Romeu,56,1393
151,Leeds United,Luke Ayling,56,1869
459,Sheffield United,Chris Basham,56,1806
194,Brighton & Hove Albion,Solly March,56,1672
275,Chelsea,N'Golo Kanté,55,1478


In [13]:
top_10_for_stat(df,"assisted_shots",15)

,team,name,assisted_shots,assisted_shots_p90,total_mins
278,Aston Villa,Jack Grealish,73,3.66,1797
346,Manchester City,Kevin De Bruyne,54,3.46,1406
57,Manchester United,Bruno Fernandes,63,3.09,1833
300,Chelsea,Mason Mount,53,2.95,1618
317,Brighton & Hove Albion,Pascal Groß,34,2.84,1078
479,Chelsea,Callum Hudson-Odoi,18,2.80,578
478,Leicester City,James Maddison,39,2.53,1385
467,Manchester City,Riyad Mahrez,29,2.52,1035
117,Everton,James Rodríguez,31,2.48,1126
392,Chelsea,Hakim Ziyech,17,2.43,630


In [14]:
top_10_for_stat(df,"assisted_shots",15,per90=False)

,team,name,assisted_shots,total_mins
278,Aston Villa,Jack Grealish,73,1797
57,Manchester United,Bruno Fernandes,63,1833
346,Manchester City,Kevin De Bruyne,54,1406
300,Chelsea,Mason Mount,53,1618
422,Wolverhampton Wanderers,Pedro Neto,42,1803
200,Liverpool,Andrew Robertson,39,1980
478,Leicester City,James Maddison,39,1385
379,Tottenham Hotspur,Son Heung-min,38,1703
273,Leeds United,Mateusz Klich,37,1664
146,Leeds United,Jack Harrison,36,1665


In [17]:
top_10_for_stat(df,"assisted_shots",10,per90=True,west_ham_only=True)

C:\Users\joeco\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,team,name,assisted_shots,assisted_shots_p90,total_mins
378,West Ham United,Ryan Fredericks,12,2.55,424
470,West Ham United,Aaron Cresswell,33,1.50,1980
434,West Ham United,Saïd Benrahma,13,1.48,788
465,West Ham United,Vladimír Coufal,25,1.39,1614
113,West Ham United,Arthur Masuaku,11,1.17,850
197,West Ham United,Michail Antonio,12,1.08,995
420,West Ham United,Jarrod Bowen,19,1.05,1621
364,West Ham United,Pablo Fornals,16,0.94,1538
97,West Ham United,Declan Rice,15,0.68,1980


In [18]:
top_10_for_stat(df,"through_balls",2,per90=False,west_ham_only=True)

,team,name,through_balls,total_mins
97,West Ham United,Declan Rice,4,1980
158,West Ham United,Andriy Yarmolenko,2,322
364,West Ham United,Pablo Fornals,2,1538
434,West Ham United,Saïd Benrahma,2,788


In [19]:
top_10_for_stat(df,"passes_completed",10,per90=False,west_ham_only=True)

,team,name,passes_completed,total_mins
470,West Ham United,Aaron Cresswell,970,1980
97,West Ham United,Declan Rice,918,1980
242,West Ham United,Tomáš Souček,637,1980
465,West Ham United,Vladimír Coufal,633,1614
118,West Ham United,Angelo Ogbonna,624,1952
364,West Ham United,Pablo Fornals,455,1538
113,West Ham United,Arthur Masuaku,388,850
216,West Ham United,Łukasz Fabiański,369,1890
420,West Ham United,Jarrod Bowen,338,1621
282,West Ham United,Fabián Balbuena,319,1080


In [20]:
top_10_for_stat(df,"pressures",10,west_ham_only=True)

C:\Users\joeco\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,team,name,pressures,pressures_p90,total_mins
373,West Ham United,Sébastien Haller,199,19.12,937
364,West Ham United,Pablo Fornals,320,18.72,1538
434,West Ham United,Saïd Benrahma,163,18.61,788
192,West Ham United,Manuel Lanzini,62,17.61,317
97,West Ham United,Declan Rice,333,15.14,1980
242,West Ham United,Tomáš Souček,328,14.91,1980
476,West Ham United,Mark Noble,38,13.67,250
158,West Ham United,Andriy Yarmolenko,47,13.13,322
465,West Ham United,Vladimír Coufal,222,12.38,1614
197,West Ham United,Michail Antonio,134,12.12,995


In [23]:
top_10_for_stat(df,"interceptions",20)

,team,name,interceptions,interceptions_p90,total_mins
275,Chelsea,N'Golo Kanté,37,2.25,1478
456,Fulham,Andre-Frank Zambo Anguissa,41,2.20,1677
84,Leeds United,Liam Cooper,29,1.96,1331
130,Manchester United,Fred,27,1.91,1272
203,Everton,Ben Godfrey,24,1.89,1140
97,West Ham United,Declan Rice,41,1.86,1980
266,Manchester United,Harry Maguire,41,1.86,1980
104,Burnley,Josh Brownhill,32,1.83,1575
152,Leicester City,Wesley Fofana,28,1.81,1391
437,Fulham,Harrison Reed,23,1.79,1159


In [26]:
top_10_for_stat(df,"nutmegs",1)

,team,name,nutmegs,nutmegs_p90,total_mins
383,Everton,Bernard,4,1.32,274
38,Leeds United,Ian Carlo Poveda,2,0.71,253
287,Manchester City,Sergio Agüero,1,0.63,143
434,West Ham United,Saïd Benrahma,5,0.57,788
89,Everton,Richarlison,7,0.50,1269
475,Sheffield United,Lys Mousset,1,0.44,203
123,Fulham,Ademola Lookman,7,0.43,1481
44,Manchester United,Marcus Rashford,8,0.41,1766
327,Wolverhampton Wanderers,Daniel Podence,6,0.41,1316
106,West Bromwich Albion,Matheus Pereira,6,0.40,1364


In [38]:
top_10_for_stat(df,"dispossessed",20)

,team,name,dispossessed,dispossessed_p90,total_mins
323,Crystal Palace,Wilfried Zaha,76,4.10,1667
331,Crystal Palace,Jordan Ayew,44,3.75,1055
188,Newcastle United,Allan Saint-Maximin,28,3.70,681
221,Leeds United,Hélder Costa,26,3.37,694
215,Crystal Palace,Jeffrey Schlupp,36,3.32,976
122,Manchester City,Raheem Sterling,55,3.24,1529
197,West Ham United,Michail Antonio,35,3.16,995
14,Crystal Palace,Michy Batshuayi,21,3.07,615
164,Liverpool,Mohamed Salah,59,3.05,1742
219,West Bromwich Albion,Grady Diangana,36,3.03,1069


In [41]:
top_10_for_stat(df,"pressure_regains",10,west_ham_only=True)

C:\Users\joeco\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,team,name,pressure_regains,pressure_regains_p90,total_mins
192,West Ham United,Manuel Lanzini,22,6.25,317
97,West Ham United,Declan Rice,104,4.73,1980
434,West Ham United,Saïd Benrahma,40,4.57,788
158,West Ham United,Andriy Yarmolenko,16,4.47,322
242,West Ham United,Tomáš Souček,90,4.09,1980
373,West Ham United,Sébastien Haller,42,4.03,937
197,West Ham United,Michail Antonio,42,3.80,995
364,West Ham United,Pablo Fornals,64,3.74,1538
113,West Ham United,Arthur Masuaku,34,3.60,850
282,West Ham United,Fabián Balbuena,41,3.42,1080


In [5]:
top_10_for_stat(df,"pressure_regains",30)

,team,name,pressure_regains,pressure_regains_p90,total_mins
332,Tottenham Hotspur,Giovani Lo Celso,42,8.61,439
173,Chelsea,Jorginho,80,8.16,882
15,Everton,Allan,89,7.79,1028
416,Brighton & Hove Albion,Alexis Mac Allister,39,7.74,454
389,Liverpool,Naby Keïta,34,7.74,395
450,Leicester City,Wilfred Ndidi,72,7.52,862
362,West Bromwich Albion,Conor Gallagher,111,7.37,1355
284,Southampton,Ibrahima Diallo,36,7.33,442
130,Manchester United,Fred,102,7.22,1272
275,Chelsea,N'Golo Kanté,118,7.19,1478
